# Codigo Final Algoritmo Evolutivo a distintos DFs- Aplicar a Base de datos Total - Reestricciones Variables especificas(Rangos) y Variables Fijas

## 1. Cargamos el Algoritmo Genetico

In [1]:
## 1. Parte 1, 

import pandas as pd
from io import BytesIO
import json

from random import Random
from time import time
from inspyred import ec
from random import Random
from time import time
import numpy as np
import tensorflow as tf
import time
 
from minio import Minio  # Cliente de Python para interactuar con el servidor MinIO.
import urllib3   # Biblioteca para realizar peticiones HTTP


# Parte 4, Algoritmo Genetico

#### Parametros a modificar en el Algoritmo Genetico,
#### window_size y   (individuo_reshape = np.array(candidate).reshape(1, 1, 14), donde modificas, (1,n_pasos_LSTM, n_variables)
#############################################################################################################################################

# # 1. Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14var_vf3.h5')

# 2. Leer el archivo Parquet en un DataFrame de pandas
# df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')
# datos_genetic = df.head(4)
# datos_genetic = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

# 3. Definir el Tamaño de la ventana (Depende con cuantos pasos atras entrenaste la LSTM, 1: solo usa 1 fila, 2: solo usa 2 filas, etc)
##### PARAMETRO A MODIFICAR
##########################################################################################################################################

window_size = 1   

# # Lista para almacenar los subconjuntos de datos que iran entregando al Algoritmo Evolutivo
# resultados = []
# # Recorrer el DataFrame usando ventanas deslizantes
# for start in range(len(bloques_origen_b1_escalado)):  # Ingresa Bloque por bloque
#     # Crear un subconjunto del DataFrame
#     subset = bloques_origen_b1_escalado.iloc[start:start + window_size].copy()
    
#     # Si el subconjunto tiene menos filas que el tamaño de la ventana, agregar filas con NaN
#     if len(subset) < window_size:
#         filas_faltantes = window_size - len(subset)
#         filas_nan = pd.DataFrame(np.nan, index=range(filas_faltantes), columns=bloques_origen_b1_escalado.columns)
#         subset = pd.concat([subset, filas_nan], ignore_index=True)
    
#     resultados.append(subset)


# Función para particionar el DataFrame
def create_sliding_windows(df, window_size):
    """
    Divide el DataFrame en ventanas deslizantes.
    
    Args:
    df (pd.DataFrame): DataFrame original.
    window_size (int): Tamaño de la ventana.
    
    Returns:
    list: Lista de DataFrames (ventanas).
    """
    windows = []
    for start in range(len(df)):
        subset = df.iloc[start:start + window_size].copy()
        
        # Rellenar con NaN si la ventana es menor al tamaño deseado
        if len(subset) < window_size:
            filas_faltantes = window_size - len(subset)
            filas_nan = pd.DataFrame(np.nan, index=range(filas_faltantes), columns=df.columns)
            subset = pd.concat([subset, filas_nan], ignore_index=True)
        
        windows.append(subset)
    return windows

###############################################################################################################################################

config_2 = {
    "dynamic_limits": {
        "roll": "fixed",  
        "intake manifold pressure": (-0.0005, 0.0005), 
        "intake manifold #2 pressure": (-0.0005, 0.0005),  
        "engine speed": (-0.209, 0.209), 
        "actual gear": (-0.3, 0.3),    
        "x": "fixed", 
        "y": "fixed",    
        "z": "fixed", 
        "efhcargado": "fixed",  
        "efhvacio": "fixed", 
        "distancia_ruta_pendiente_grados": "fixed",    
        "ambient air temperature": "fixed", 
        "brake pump pressure": (-0.0766, 0),    
        "fuel consumption rate(lag1)": "fixed"    
    }
}

def generate_dynamic_limits(row, config):
    """
    Genera X_min y X_max para una fila específica basada en configuraciones personalizadas.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    config (dict): Configuración de límites dinámicos por columna.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    #Estas listas se llenarán con los límites mínimos (X_min) y máximos (X_max) para cada valor en la fila procesada
    X_min = []
    X_max = []
    
    for i, value in enumerate(row): # enumerate, nos permite iterar sobre los valores de la fila (row) y obtener tanto el índice i como el valor value
        variable_name = config["column_names"][i]  # config["column_names"] contiene los nombres de las columnas en el orden en que aparecen en el DataFrame.
        
        # Si el nombre de la columna tiene la etiqueta "fixed" en config["dynamic_limits"], la variable es fija y no debe modificarse
        if config["dynamic_limits"].get(variable_name) == "fixed":  # 
            # Variable fija, los límites son el mismo valor
            X_min.append(value)
            X_max.append(value)
        else:
            # Aplicar los límites dinámicos definidos
            lower, upper = config["dynamic_limits"][variable_name]

            # Si value = 0.1 y lower = -0.2: value+lower=0.1−0.2=−0.1, new_min=max⁡(0,−0.1)=0, y asi Evitamos negativos en valores escalados 
            new_min = max(0, value + lower)  #  # Evitar negativos en valores escalados , en este caso minimo es 0, porque es max-minescaler, en otro escalador puede ser -1

            # Si value = 0.8 y upper = 0.3: value+upper=0.8+0.3=1.1, new_max=min⁡(1,1.1)=1, y asi evitamos valores por encima de 1 en valores escalados
            new_max = min(1, value + upper)  # Limitar al rango [0, 1] escalado, en este caso minimo es 1, porque es max-minescaler, en otro escalador puede ser 1, etc

            
            # Obtiene los valores lower (límite inferior) y upper (límite superior) definidos en el diccionario de configuración para la columna actual (variable_name).
            # lower, upper = config["dynamic_limits"][variable_name]
            # new_min = max(0, value + lower)  # Evitar valores negativos
            # new_max = max(new_min, value + upper)  # Asegurar que X_max >= X_min
            
            # X_min.append(value + lower)
            # X_max.append(value + upper)
            X_min.append(new_min)
            X_max.append(new_max)
    
    return np.array(X_min), np.array(X_max)


def generate_individual(random, df, start_index, n_var_independientes, ventana, config):
    """
    Genera un individuo basado en configuraciones personalizadas.
    asegurándose de que no se generen valores negativos.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    ventana (int): Tamaño de la ventana (número de filas a seleccionar).
    config (dict): Configuración personalizada para los límites dinámicos.
    
    Returns:
    individual (array): Array de tamaño `(ventana, n_var_independientes)` con los valores generados.
    """
    individual = []

    # Asegurarse de trabajar solo con las primeras `n_var_independientes` columnas(No debemos incluir la columna Target: En este caso la ultima columna)
    df_independientes = df.iloc[:, :n_var_independientes]  # Selecciona únicamente las primeras n_var_independientes columnas del DataFrame 
    config["column_names"] = df_independientes.columns.tolist() # Guarda los nombres de estas columnas en config["column_names"] para usarlos más adelante en la generación de límites dinámicos.

    # Verificar que el índice de inicio + ventana no exceda el número de filas en df_independientes
    if start_index + ventana > len(df_independientes):
        raise IndexError(f"El índice de inicio más la ventana excede el número de filas disponibles en el DataFrame (inicio: {start_index}, ventana: {ventana}).")
    
    # Iterar a través de la ventana
    for i in range(ventana): # Itera sobre las filas consecutivas del DataFrame, comenzando desde start_index
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row, config = config)

        # Generar una fila del individuo respetando los límites
        # random_row = [random.uniform(X_min[j], X_max[j]) for j in range(len(row))]

        # Generar una fila del individuo respetando los límites
        random_row = []
        for j in range(len(row)):
            value = random.uniform(X_min[j], X_max[j])  # Generar valor dentro de los límites
            if value < 0:  # Si el valor es negativo, conservar el original del individuo original del df, como ya hemos limitado en
                # generate_dynamic_limits() con new_min = max(0, value + lower), esta linea podria ser omitida, pero conservemosla
                value = row[j]
            random_row.append(value)
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)


# import random
# window_size = 1
# individuo = generate_individual(random, resultados[10], start_index=0, n_var_independientes=14, ventana=window_size, config = config_2)
# individuo

def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 1, 14)  #  (1, n_pasos_lstm, n_variables) , n_pasos_lstm >> PARAMETRO A MODIFICAR
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")\

# Función de evaluación y algoritmo genético
# window_size, Tamaño de la ventana (Depende con cuantos pasos atras entrenaste la LSTM, 1: solo usa 1 fila, 2: solo usa 2 filas, etc)

def run_genetic_algorithm(modelo_lstm, df, n_individuos_poblacion=5, generations=5, start_index=0, window_size=1): ##### window_size , PARAMETRO A MODIFICAR
    # Crear el generador de números aleatorios
    prng = Random()
    
    # Número de variables independientes en cada individuo
    n_var_independientes = 14  
    
    # Crear el objeto del algoritmo genético (GA)
    ea = ec.GA(prng)

    # Configurar la función generadora de individuos
    def generator(random, args):
        #return generate_individual(prng, df, start_index, n_var_independientes, window_size)
        #return generate_individual(random, df, start_index=0, n_var_independientes=14, ventana=window_size)
        return generate_individual(random, df, start_index=0, n_var_independientes=14, ventana=window_size, config = config_2)
    
    # Crear un contenedor global para almacenar los fitness de cada generación
    fitness_history = {}

###################################################################################################################################################
    # # Configurar la función de evaluación (fitness)
    # def evaluator(candidates, args):
    #     # Evaluar el fitness de todos los candidatos (individuos)
    #     fitness_values = fitness_function(candidates, args, modelo_lstm)
        
    #     # Obtener el número de generación actual
    #     current_generation = len(fitness_history)
    #     #print(f"Evaluando Generación {current_generation}...")
        
    #     # Guardar los fitness de la generación actual
    #     fitness_history[current_generation] = fitness_values
        
    #     # Mostrar los fitness de la generación actual
    #     #print(f"\nFitness de la Generación {current_generation}: {fitness_values}")
        
    #     return fitness_values

#########################################################################################################################################################
    # Límite de estancamiento
    stagnation_limit = 10
    stagnation_counter = 0
    last_best_fitness = None
    
    # Configurar la función de evaluación (fitness)
    def evaluator(candidates, args):
        nonlocal stagnation_counter, last_best_fitness
    
        # Evaluar el fitness de todos los candidatos (individuos)
        fitness_values = fitness_function(candidates, args, modelo_lstm)
        
        # Obtener el número de generación actual
        current_generation = len(fitness_history)
        
        # Guardar los fitness de la generación actual
        fitness_history[current_generation] = fitness_values
    
        # Calcular el mejor fitness de la generación actual
        current_best_fitness = min(fitness_values)  # Asumes que estás minimizando
        
        # Verificar estancamiento
        if last_best_fitness is not None and current_best_fitness == last_best_fitness:
            stagnation_counter += 1
        else:
            stagnation_counter = 0
    
        # Detener si se alcanza el límite de estancamiento
        if stagnation_counter >= stagnation_limit:
            print(f"Estancamiento detectado en la Generación {current_generation}. Mejor fitness: {current_best_fitness}")
            raise StopIteration("El algoritmo se detuvo debido a estancamiento.")
    
        # Actualizar el último mejor fitness
        last_best_fitness = current_best_fitness
    
        # Mostrar los fitness de la generación actual
        # print(f"Generación {current_generation} - Mejor fitness: {current_best_fitness}")
    
        return fitness_values

#########################################################################################################################################################
    

    # Configuración de operadores genéticos
    ea.variator = [
        ec.variators.uniform_crossover #,  # Cruce uniforme
        #ec.variators.gaussian_mutation   # Mutación gaussiana,  En este caso, no estoy usando Mutacion
    ]

    #ea.variator = [ec.variators.uniform_crossover, custom_mutation]  # Usar custom_mutation

    # Cambiar el método de reemplazo a generational_replacement
    ea.replacer = ec.replacers.generational_replacement
    ea.selector = ec.selectors.tournament_selection  # Selección por torneo

    # Configurar la terminación por número de generaciones
    ea.terminator = ec.terminators.generation_termination
        
    # Ejecutar la evolución
    final_pop = ea.evolve(
        generator=generator,
        evaluator=evaluator,
        pop_size=n_individuos_poblacion,
        maximize=False, #Problema de Minimizacion 
        num_elites=1,
        max_generations=generations
    )

    # Obtener el mejor individuo
    best_individual = min(final_pop, key=lambda x: x.fitness)
    best_fitness = best_individual.fitness

    print("\nMejor individuo (minimizar consumo de combustible):", best_individual.candidate)
    print("Consumo de combustible predicho (fitness):", best_fitness)

    return best_individual.candidate, best_fitness


#Función para aplicar el algoritmo genético por bloques
#window_size, Tamaño de la ventana (Depende con cuantos pasos atras entrenaste la LSTM, 1: solo usa 1 fila, 2: solo usa 2 filas, etc)
def aplicar_algoritmo_genetico_por_bloques(modelo_lstm, resultados, window_size=1, n_individuos_poblacion=5, generations=5): ##### window_size , PARAMETRO A MODIFICAR
    # Nombres de columnas originales del DataFrame
    columnas_originales = resultados[0].columns.tolist()  # Aseguramos que incluya todas las columnas de resultados[0]
    
    # Crear un DataFrame vacío para almacenar los resultados
    df_resultados_final = pd.DataFrame(columns=columnas_originales) 
    
    # Iterar sobre los bloques en resultados con una ventana deslizante
    for i in range(len(resultados) - window_size + 1):  # Ajustamos el rango para no exceder el tamaño del DataFrame
        # Ejecutar el algoritmo genético en el bloque actual, pasando el índice de inicio como `start_index`
        best_individual, best_fitness = run_genetic_algorithm(
            modelo_lstm, resultados[i], n_individuos_poblacion=n_individuos_poblacion, generations=generations, start_index=i, window_size=window_size)
        
        # Extraer solo la última fila de best_individual (la última fila de la matriz 4x14)
        ultima_fila = best_individual[-1]  # Esto obtiene la última fila del array
        
        # Crear una nueva fila con los valores de ultima_fila y best_fitness
        nueva_fila = list(ultima_fila)  # Convertimos la última fila en una lista
        nueva_fila.append(best_fitness)  # Agregamos el best_fitness al final
        
        # Convertimos la fila en un DataFrame temporal
        df_temp = pd.DataFrame([nueva_fila], columns=columnas_originales) 
        
        # Anexamos la fila al DataFrame final
        df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)

        # Ajustar el índice del DataFrame final para que comience en window_size - 1
        df_resultados_final.index = range(window_size - 1, window_size - 1 + len(df_resultados_final))

    return df_resultados_final

## 2. Cargamos los Equipos(Camiones) que queremos procesar

In [2]:
from conection_storage import AzureBlobStorage
import pandas as pd

# Definir las columnas necesarias
columnas_necesarias = [
    'roll',
    'intake manifold pressure',
    'intake manifold #2 pressure',
    'engine speed',
    'actual gear',
    'x',
    'y',
    'z',
    'efhcargado',
    'efhvacio',
    'distancia_ruta_pendiente_grados',
    'ambient air temperature',
    'brake pump pressure',
    'fuel consumption rate(lag1)', 
    'fuel consumption rate(target)'
]

# Configuración de la conexión a Azure Blob Storage
config_storage = {
    'accountname': 'datalakemlopsd4m',
    'accountkey': 'iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==',
}

# Conectar al servidor de AzureStorage
storage = AzureBlobStorage(**config_storage)

# Definir las rutas de los archivos Parquet
files_to_process = [
    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/28/',
        'filename': '20241203120452_Caso_1.parquet'
    },
    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/29/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/30/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/31/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/32/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/44/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/45/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/49/',
        'filename': '20241203120452_Caso_1.parquet'
    },
    

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/183/',
        'filename': '20241203120452_Caso_1.parquet'
    },

    {
        'container': 'presentation',
        'folder': 'proyectocombustible_presentation/ModeloDinamico/datacamion/186/',
        'filename': '20241203120452_Caso_1.parquet'
    },
    
    # Agrega más archivos según sea necesario
]

# Diccionario para almacenar cada DataFrame por archivo
dataframes_dict = {}

# Leer cada archivo Parquet y guardarlo en el diccionario
for idx, file_info in enumerate(files_to_process, start=1):
    container_name = file_info['container']
    full_path = file_info['folder'] + file_info['filename']
    print(f"Leyendo archivo: {full_path}")
    
    # Leer el archivo desde Azure Storage
    df = storage.read_file(container_name, full_path, 'parquet')
    
    # Ordenar por la columna 'instant_date_t'
    df_sorted = df.sort_values('instant_date_t', ignore_index=True)

    # Filtrar columnas necesarias
    df_sorted = df_sorted[columnas_necesarias]
    
    # Guardar en el diccionario usando el índice o nombre del archivo
    key = f"df_{file_info['folder']}"  #f"df_{idx}"  # O usa file_info['filename'] si prefieres nombres descriptivos
    dataframes_dict[key] = df_sorted   # organiza y almacena cada DataFrame ordenado para su posterior procesamiento. 

# Reemplazar los valores NaN por 0 en cada DataFrame del diccionario
for key, df in dataframes_dict.items():
    print(f"Reemplazando NaN por 0 en el DataFrame: {key}")
    
    # Reemplazar los valores NaN por 0
    df_filled = df.fillna(0)
    
    # Guardar el DataFrame con los valores NaN reemplazados
    dataframes_dict[key] = df_filled

    # Verificar el resultado
    print(f"DataFrame después de reemplazar NaN por 0 en {key}:")
    #print(df_filled.head())  # Mostrar las primeras filas para verificar

Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/28/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/29/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/30/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/31/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/32/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/44/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/45/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/49/20241203120452_Caso_1.parquet
Leyendo archivo: proyectocombustible_presentation/ModeloDinamico/datacamion/183/20241203120452_Caso_1.parquet
Leyendo archivo: p

In [166]:
# len(dataframes_dict['df_proyectocombustible_presentation/ModeloDinamico/datacamion/28/'])

In [3]:
def dividir_en_bloques(df, num_bloques=15):
    """
    Divide un DataFrame en bloques del mismo tamaño.
    Args:
        df (pd.DataFrame): DataFrame a dividir.
        num_bloques (int): Número de bloques a crear.
    Returns:
        dict: Diccionario con bloques numerados como claves.
    """
    bloque_sizes = [len(df) // num_bloques + (1 if x < len(df) % num_bloques else 0) for x in range(num_bloques)]
    bloques = {}
    start_idx = 0

    for i, size in enumerate(bloque_sizes):
        end_idx = start_idx + size
        bloques[f'bloque_{i + 1}'] = df.iloc[start_idx:end_idx]
        start_idx = end_idx

    return bloques

In [4]:
# # Variables a normalizar (excluyendo la última columna)
# variables_a_normalizar = bloques_origen['bloque_1'].columns[:-2]  ##  Queda fijo 

# # Excluir el target y el lag, que no se deben escalar
# with open("minimos_lstm_13_var.json", "r") as file:
#     minimos = json.load(file)

# with open("maximos_lstm_13_var.json", "r") as file:
#     maximos = json.load(file)

def escalar_por_variables_max_min(dataframe, variables_a_normalizar, minimos, maximos):
    """
    Normaliza solo las variables especificadas en un dataframe usando el método Min-Max.
    Args:
        dataframe: El dataframe que se va a normalizar.
        variables_a_normalizar: Lista de nombres de las variables a normalizar.
        minimos: Diccionario con el valor mínimo de cada variable.
        maximos: Diccionario con el valor máximo de cada variable.
    Returns:
        DataFrame normalizado.
    """
    dataframe_normalizado = dataframe.copy()
    for columna in variables_a_normalizar:
        if columna in minimos and columna in maximos:
            dataframe_normalizado[columna] = (dataframe[columna] - minimos[columna]) / (maximos[columna] - minimos[columna])
    return dataframe_normalizado

In [5]:
# Función principal de procesamiento
def procesar_dataframes(dataframes_dict, modelo_lstm, minimos_path, maximos_path, window_size=1, n_individuos_poblacion=5, generations=5):
    """
    Procesa múltiples DataFrames dividiéndolos en bloques, normalizándolos, aplicando un algoritmo genético a cada bloque,
    y consolidando los resultados en un DataFrame final para cada DataFrame original.

    Args:
        dataframes_dict (dict): Diccionario con los DataFrames a procesar.
        modelo_lstm: Modelo LSTM ya entrenado para el algoritmo genético.
        minimos_path (str): Ruta al archivo JSON con los valores mínimos.
        maximos_path (str): Ruta al archivo JSON con los valores máximos.
        window_size (int): Tamaño de la ventana deslizante.
        n_individuos_poblacion (int): Número de individuos en la población del algoritmo genético.
        generations (int): Número de generaciones del algoritmo genético.

    Returns:
        dict: Diccionario con los DataFrames procesados y consolidados.
    """
    # Cargar los valores mínimos y máximos
    with open(minimos_path, "r") as file:
        minimos = json.load(file)
    with open(maximos_path, "r") as file:
        maximos = json.load(file)

    # Diccionario para almacenar los resultados finales
    recomendaciones_dict = {}

    # Número de bloques a dividir
    num_bloques = 15


    # Recorrer cada DataFrame en el diccionario
    for key, df in dataframes_dict.items():
        print(f"\nProcesando el DataFrame {key}...")

        # Dividir el DataFrame en bloques
        bloques_origen = dividir_en_bloques(df, num_bloques)

        # Determinar las variables a normalizar (excluyendo las dos últimas columnas)
        variables_a_normalizar = bloques_origen['bloque_1'].columns[:-2]

        # Normalizar cada bloque
        for bloque_key, bloque_df in bloques_origen.items():
            bloques_origen[bloque_key] = escalar_por_variables_max_min(
                bloque_df, variables_a_normalizar, minimos, maximos
            )

        # Lista para consolidar los resultados de los bloques
        df_resultados_bloques = []

        # Procesar cada bloque individualmente
        for bloque_key, bloque_df in bloques_origen.items():
            print(f"  Procesando {bloque_key} con {len(bloque_df)} filas...")

            # Crear ventanas deslizantes para el bloque
            resultados = create_sliding_windows(bloque_df, window_size)

            # Medir el tiempo de ejecución del algoritmo genético
            start_time = time.time()

            # Aplicar el algoritmo genético al bloque
            df_resultados_recomendacion = aplicar_algoritmo_genetico_por_bloques(
                modelo_lstm, resultados, window_size=1, n_individuos_poblacion=5, generations=5)
            
            end_time = time.time()

            # Agregar los resultados del bloque a la lista
            df_resultados_bloques.append(df_resultados_recomendacion)

            print(f"    {bloque_key} procesado en {end_time - start_time:.2f} segundos.")

        # Consolidar los resultados de todos los bloques para el DataFrame actual
        df_recomendacion = pd.concat(df_resultados_bloques, ignore_index=True)

        # Almacenar el resultado consolidado
        recomendaciones_dict[f"{key}_recomendacion"] = df_recomendacion

        print(f"DataFrame {key} procesado y consolidado.")

    return recomendaciones_dict

In [ ]:
#Ejemplo de uso:
# Medir el tiempo de ejecución del algoritmo genético
start_time = time.time()

recomendaciones = procesar_dataframes(dataframes_dict,modelo_lstm,"minimos_lstm_13_var.json","maximos_lstm_13_var.json",window_size=1,n_individuos_poblacion=5,generations=5)

end_time = time.time()
# Calcular y mostrar el tiempo de ejecución
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución del algoritmo genético: {elapsed_time:.4f} segundos")


Procesando el DataFrame df_proyectocombustible_presentation/ModeloDinamico/datacamion/28/...
  Procesando bloque_1 con 28 filas...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16m

/tmp/ipykernel_3634220/2923353968.py:370: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

Mejor individuo (minimizar consumo de combustible): [[5.94569623e-01 9.77257043e-01 2.15617159e-02 3.25472581e-01
  3.11967351e-01 9.82235754e-03 9.99839779e-03 9.

/tmp/ipykernel_3634220/2923353968.py:370: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

Mejor individuo (minimizar consumo de combustible): [[5.13113260e-01 9.77020840e-01 2.14555546e-02 1.58030641e-01
  3.24835372e-01 9.94995509e-03 9.99741988e-03 9.

/tmp/ipykernel_3634220/2923353968.py:370: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

Mejor individuo (minimizar consumo de combustible): [[5.74514091e-01 9.77541951e-01 1.22566271e-02 7.54536870e-01
  5.82938167e-01 9.97656694e-03 9.99683028e-03 9.

/tmp/ipykernel_3634220/2923353968.py:370: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

Mejor individuo (minimizar consumo de combustible): [[6.79419935e-01 9.77707109e-01 6.01767232e-02 7.36381568e-01
  9.87557896e-01 9.71190332e-03 9.99633552e-03 9.

In [207]:
# recomendaciones['df_proyectocombustible_presentation/ModeloDinamico/datacamion/45/_recomendacion'].head()

In [ ]:
## Parte 5, Desescalamos los datos

In [192]:
# Parte 5, Desescalamos los datos 

def desescalar_por_variables_max_min(dataframe_normalizado, variables_a_desescalar, minimos, maximos):
    """
    Desescala solo las variables especificadas en un dataframe utilizando el método inverso de Min-Max.
    Args:
        dataframe_normalizado: El dataframe con valores normalizados.
        variables_a_desescalar: Lista de nombres de las variables a desescalar.
        minimos: Diccionario con el valor mínimo de cada variable.
        maximos: Diccionario con el valor máximo de cada variable.
    Returns:
        DataFrame desescalado con valores originales.
    """
    dataframe_desescalado = dataframe_normalizado.copy()
    for columna in variables_a_desescalar:
        if columna in minimos and columna in maximos:
            dataframe_desescalado[columna] = dataframe_normalizado[columna] * (maximos[columna] - minimos[columna]) + minimos[columna]
    return dataframe_desescalado



# Función para desescalar múltiples DataFrames
def desescalar_recomendaciones(recomendaciones_dict, variables_a_desescalar, minimos, maximos):
    """
    Desescala todas las recomendaciones dentro de un diccionario de DataFrames.
    Args:
        recomendaciones_dict: Diccionario con los DataFrames de recomendaciones.
        variables_a_desescalar: Lista de nombres de las variables a desescalar.
        minimos: Diccionario con los valores mínimos por variable.
        maximos: Diccionario con los valores máximos por variable.
    Returns:
        Diccionario con los DataFrames desescalados.
    """
    recomendaciones_desescaladas = {}
    
    for key, dataframe in recomendaciones_dict.items():
        print(f"Desescalando el DataFrame: {key}")
        dataframe_desescalado = desescalar_por_variables_max_min(
            dataframe,
            variables_a_desescalar,
            minimos,
            maximos
        )
        recomendaciones_desescaladas[key] = dataframe_desescalado
    
    return recomendaciones_desescaladas


with open("minimos_lstm_13_var.json", "r") as f:
    minimos = json.load(f)

with open("maximos_lstm_13_var.json", "r") as f:
    maximos = json.load(f)

variables_a_desescalar = list(minimos.keys())  # Todas las variables en 'minimos' deben ser desescaladas.

# Desescalar los DataFrames
recomendaciones_desescaladas = desescalar_recomendaciones(recomendaciones,variables_a_desescalar,minimos,maximos)

Desescalando el DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/28/_recomendacion
Desescalando el DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/29/_recomendacion
Desescalando el DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/44/_recomendacion
Desescalando el DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/45/_recomendacion


In [204]:
# dataframes_dict['df_proyectocombustible_presentation/ModeloDinamico/datacamion/28/'].head()

In [205]:
# recomendaciones_desescaladas['df_proyectocombustible_presentation/ModeloDinamico/datacamion/28/_recomendacion'].head()

In [206]:
# dataframes_recomendaciones_actualizados['df_proyectocombustible_presentation/ModeloDinamico/datacamion/45/'].head()

In [196]:
# # Parte 6, Aplicar recomendaciones finales del Algoritmo Evolutivo
def aplica_recomendaciones_finales(df_original, df_desescalado, columna_objetivo, columna_redondeo):
    """
    Aplica las recomendaciones basadas en una condición y actualiza las filas del DataFrame original con el DataFrame desescalado.
    
    Parameters:
    df_original (pd.DataFrame): DataFrame original con los datos base.
    df_desescalado (pd.DataFrame): DataFrame con los valores desescalados.
    columna_objetivo (str): Nombre de la columna objetivo para la comparación.
    columna_redondeo (str): Nombre de la columna que se debe redondear al final.

    Returns:
    pd.DataFrame: DataFrame actualizado con las recomendaciones aplicadas.
    """
    # Asegurar que los índices son idénticos
    df_original = df_original.reset_index(drop=True)
    df_desescalado = df_desescalado.reset_index(drop=True)
    
    # Crear una copia del DataFrame original para modificar
    df_final = df_original.copy()
    
    # Condición de comparación
    condicion = df_desescalado[columna_objetivo] < df_original[columna_objetivo]
    
    # Actualizar las filas donde se cumple la condición
    df_final.loc[condicion] = df_desescalado.loc[condicion]
    
    # Redondear la columna especificada
    df_final[columna_redondeo] = df_final[columna_redondeo].round()
    
    return df_final


def aplicar_recomendaciones_a_dataframes(dataframes_dict, recomendaciones_desescaladas, columna_objetivo, columna_redondeo):
    """
    Aplica recomendaciones finales a múltiples DataFrames originales basándose en sus versiones desescaladas.
    
    Parameters:
    dataframes_dict (dict): Diccionario con los DataFrames originales.
    recomendaciones_desescaladas (dict): Diccionario con los DataFrames recomendados desescalados.
    columna_objetivo (str): Nombre de la columna objetivo para comparación.
    columna_redondeo (str): Nombre de la columna que se debe redondear.

    Returns:
    dict: Diccionario con los DataFrames actualizados.
    """
    dataframes_actualizados = {}

    for key, df_original in dataframes_dict.items():
        # Verificar si existe un DataFrame desescalado correspondiente
        recomendacion_key = f"{key}_recomendacion"
        if recomendacion_key in recomendaciones_desescaladas:
            print(f"Aplicando recomendaciones al DataFrame: {key}")
            
            # Obtener el DataFrame recomendado desescalado
            df_desescalado = recomendaciones_desescaladas[recomendacion_key]
            
            # Aplicar las recomendaciones
            df_actualizado = aplica_recomendaciones_finales(
                df_original,
                df_desescalado,
                columna_objetivo,
                columna_redondeo
            )
            
            # Guardar el DataFrame actualizado
            dataframes_actualizados[key] = df_actualizado
        else:
            print(f"No se encontró recomendación para el DataFrame: {key}")

    return dataframes_actualizados

In [197]:
columna_objetivo = "fuel consumption rate(target)"  # Ejemplo de columna para comparación
columna_redondeo = "actual gear"  # Ejemplo de columna a redondear

# Aplicar recomendaciones a todos los DataFrames originales
dataframes_recomendaciones_actualizados = aplicar_recomendaciones_a_dataframes(dataframes_dict, recomendaciones_desescaladas, columna_objetivo, columna_redondeo)

Aplicando recomendaciones al DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/28/
Aplicando recomendaciones al DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/29/
Aplicando recomendaciones al DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/44/
Aplicando recomendaciones al DataFrame: df_proyectocombustible_presentation/ModeloDinamico/datacamion/45/


/tmp/ipykernel_120300/1538960697.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-2.01999988e+00 -1.72999815e+00 -1.69999944e+00  6.15000080e+00
 -2.82999993e+00 -3.00008418e-02 -1.66999879e+00 -8.49999816e-01
  1.75000141e+00  9.10000715e-01 -1.10000005e+00 -6.61000015e+00
 -6.07999905e+00 -3.96000043e+00  5.05000095e+00 -2.03000074e+00
  2.05000014e+00  6.82000041e+00  6.05000187e+00  2.36000166e+00
  2.70001751e-01 -5.69000031e+00 -5.61000020e+00  1.11000051e+00
 -5.49999155e-01 -4.28999787e+00 -3.02999973e+00 -3.02000080e+00
 -1.46000007e+00 -1.39999877e+00 -3.55999890e+00 -1.08999919e+00
 -8.30000029e-01 -1.96107010e-07  1.16000094e+00  6.06999973e+00
 -7.29999955e+00 -6.52000014e+00 -7.11000061e+00 -7.01999964e+00
  2.15999993e+00  1.78000205e+00  2.91000062e+00  4.43000178e+00
 -1.63999814e+00 -7.35999987e+00  2.02000142e+00  1.96999906e+00
 -6.77000037e+00 -8.10999959e+00 -5.15999921e+00  1

In [203]:
dataframes_recomendaciones_actualizados['df_proyectocombustible_presentation/ModeloDinamico/datacamion/45/'].head()

,roll,intake manifold pressure,intake manifold #2 pressure,engine speed,actual gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,ambient air temperature,brake pump pressure,fuel consumption rate(lag1),fuel consumption rate(target)
0,1.810000,1.890000,3.200000,1557.500000,2.0,204321.60,8397120.77,4221.5,1710833.0,1278555.0,-0.028648,13.000000,15668.000000,4.000000,100.331581
1,-1.770000,20.237151,30.416329,1158.731379,0.0,204731.21,8396733.06,4183.8,1710833.0,1278555.0,-0.028648,12.889963,257.343881,100.331581,221.482224
2,-0.950001,132.879989,147.625971,1163.732272,4.0,204545.22,8397074.03,4226.9,1710833.0,1278555.0,-0.028648,12.688687,356.144701,247.209137,303.497192
3,-0.729999,150.275126,161.168630,1949.810027,-1.0,204797.90,8396937.66,4255.2,1710833.0,1278555.0,-0.028648,12.090720,291.851396,320.766815,333.605377
4,-2.729999,173.891287,157.931484,1344.087129,3.0,204626.50,8397229.17,4299.3,1710833.0,1278555.0,-0.028648,12.090720,375.300850,336.619781,334.939087
